In [1]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [2]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA_sim_body_STLs/qrpfra_v3_leg_ik_scene_right.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        x_position_before = self.data.qpos[0]

        self.do_simulation(action, self.frame_skip)
        x_position_after = self.data.qpos[0]

        observation = self._get_obs()
        reward = x_position_after - x_position_before
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [3]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([ 0.045 ,  0.1335, -0.107 ,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [4]:
left_leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [5]:
for i in range(-100, 100):
    for j in range(-100, 100):
        for k in range(-100, 100):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,5):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 4:
                    env_obs = obs

            left_leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])

            #print(obs)
            if done:
                env.reset()

env.close()

In [6]:
print(len(left_leg_dataset))

8000000


In [7]:
#Convert left leg list to numpy array
left_leg_dataset_as_np = np.array(left_leg_dataset)

In [11]:
print(left_leg_dataset_as_np[0:4])

observations = left_leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = left_leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.17804887 -0.14482009 -0.04103864 -1.         -1.         -1.        ]
 [ 0.17749582 -0.14609836 -0.03892726 -1.         -1.         -0.99      ]
 [ 0.17839362 -0.14422172 -0.04031275 -1.         -1.         -0.98      ]
 [ 0.17893699 -0.14308035 -0.04067234 -1.         -1.         -0.97      ]]
[[ 0.17804887 -0.14482009 -0.04103864]
 [ 0.17749582 -0.14609836 -0.03892726]
 [ 0.17839362 -0.14422172 -0.04031275]
 ...
 [ 0.06803363  0.0427334  -0.0018736 ]
 [ 0.06751309  0.04162337 -0.00217766]
 [ 0.0669825   0.04051974 -0.0024876 ]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 0.99  0.99  0.97]
 [ 0.99  0.99  0.98]
 [ 0.99  0.99  0.99]]


In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='tanh')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_obs, train_actions, epochs=10, batch_size=1024)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


Epoch 1/10
7032/7032 [==============================] - 7s 956us/step - loss: 0.0267 - accuracy: 0.9205
Epoch 2/10
7032/7032 [==============================] - 7s 952us/step - loss: 0.0174 - accuracy: 0.9423
Epoch 3/10
7032/7032 [==============================] - 7s 950us/step - loss: 0.0168 - accuracy: 0.9445
Epoch 4/10
7032/7032 [==============================] - 7s 955us/step - loss: 0.0165 - accuracy: 0.9452
Epoch 5/10
7032/7032 [==============================] - 7s 961us/step - loss: 0.0162 - accuracy: 0.9455
Epoch 6/10
7032/7032 [==============================] - 7s 982us/step - loss: 0.0159 - accuracy: 0.9458
Epoch 7/10
7032/7032 [==============================] - 7s 972us/step - loss: 0.0156 - accuracy: 0.9462
Epoch 8/10
7032/7032 [==============================] - 7s 983us/step - loss: 0.0154 - accuracy: 0.9460
Epoch 9/10
7032/7032 [==============================] - 7s 993us/step - loss: 0.0152 - accuracy: 0.9458
Epoch 10/10
25000/25000 [==============================] - 8s 33

In [13]:
model.save("right_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: right_legs_model/assets


INFO:tensorflow:Assets written to: right_legs_model/assets
